## The prediction function

This function takes simple inputs to return the predicted number of rides.

In [1]:
import joblib
import pandas as pd
from tranquilizer import tranquilize

In [2]:
model = joblib.load('model.pkl')

The `tranquilize` decorator will mark the function for the tranquilizer command-line tool to serve as a REST API endpoint.

The *docstrings* become the description of the endpoint in the served Swagger documentation.

In [3]:
@tranquilize(method='GET')
def params() -> dict:
    """Return the parameters used to train the model"""
    
    return model.named_steps['gradientboostingregressor'].get_params()

In [4]:
params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 800,
 'n_iter_no_change': 5,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.01,
 'validation_fraction': 0.2,
 'verbose': 0,
 'warm_start': False}

Type hints are not required, but add input validation and data type convesion when a request is made. By default Tranquilizer assumes inputs are all strings.

In [5]:
@tranquilize(method='POST')
def predict(max_temp: float, month: str, weekend: bool, holiday: bool) -> int:
    """Predict the number of bikeshare rides in Pittsburgh
    
    :param max_temp: The forecasted maximum temperature in degrees Fahrenheit
    :param month: The month (i.e, January)
    :param holiday: Is today a holiday?
    :param weekend: Is today a weekend?
    """

    df = pd.DataFrame({
        'max_temp': max_temp,
        'month': month,
        'holiday': holiday,
        'weekend': weekend
    }, index=[0])
    
    rides = model.predict(df)[0]
    
    return int(rides)

Our Python function hasn't changed and we can still run it here in the notebook.

In [6]:
predict(85.0, 'July', True, False)

544

Finally, launch the Tranquilizer server

In [7]:
!tranquilizer --name Bikeshare predict.ipynb

 * Serving Flask app "tranquilizer.application" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8086/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2021 19:32:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2021 19:32:23] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2021 19:32:23] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2021 19:32:23] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2021 19:32:23] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2021 19:32:24] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2021 19:32:24] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 304 -
^C
